In [ ]:
import pandas as pd
data1 = pd.read_csv('./01_제공데이터_update(210811)/01_음식물쓰레기_FOOD_WASTE_210811_update.csv', encoding='cp949')
data2 = pd.read_csv('./01_제공데이터_update(210811)/02-1_내국인유동인구_KOREAN.csv', encoding='cp949')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
def get_year_month(x):
    return x[0:7]

In [ ]:
data1['year_month'] = data1['base_date'].apply(lambda x: get_year_month(x))
data2['year_month'] = data2['base_date'].apply(lambda x: get_year_month(x))

In [ ]:
temp1 = data1.copy()
temp1 = temp1.groupby(['emd_nm', 'year_month']).sum().reset_index()
temp2 = data2.copy()
temp2 = data2.groupby(['emd_nm', 'year_month', 'time', 'age']).sum().reset_index()

In [ ]:
temp2_prep = temp2[['emd_nm', 'year_month']].drop_duplicates().reset_index(drop=True)
for i in sorted(set(temp2['age'])):
    temp = temp2[temp2['age'] == i]
    for j in sorted(set(temp['time'])):
        temp_1 = temp[temp['time'] == j].reset_index()
        exec("temp2_prep['r-"+ str(i) + "-" + str(j) +"'] = temp_1['resd_pop_cnt']")

for i in sorted(set(temp2['age'])):
    temp = temp2[temp2['age'] == i]
    for j in sorted(set(temp['time'])):
        temp_1 = temp[temp['time'] == j].reset_index()
        exec("temp2_prep['w-"+ str(i) + "-" + str(j) +"'] = temp_1['work_pop_cnt']")
        
for i in sorted(set(temp2['age'])):
    temp = temp2[temp2['age'] == i]
    for j in sorted(set(temp['time'])):
        temp_1 = temp[temp['time'] == j].reset_index()
        exec("temp2_prep['v-"+ str(i) + "-" + str(j) +"'] = temp_1['visit_pop_cnt']")
        

In [ ]:
temp_var = list(temp2_prep.columns)[2:]
emd_var = list(set(temp2_prep.emd_nm))

In [ ]:
pd.DataFrame(columns=temp_var, index=['raw', 'diff', 'log', 'log_diff', 'method'])

In [ ]:
def adf_test(x):
    if sum(1-np.isfinite(x)) == 0:
        return adfuller(x)[1]
        
    else:
        return np.nan

In [ ]:
from statsmodels.tsa.stattools import adfuller
n=0
for i in sorted(set(temp2_prep['emd_nm'])):
    temp = temp2_prep[temp2_prep['emd_nm'] == i].reset_index(drop=True)
    exec("method_" + str(n) + "= pd.DataFrame(columns=temp_var, index=['raw', 'diff', 'log', 'log_diff', 'method'])")
    exec("method_" + str(n) + ".loc['raw',:] = list(temp.iloc[:,2:].apply(lambda x: adf_test(x), axis=0))")
    exec("method_" + str(n) + ".loc['log',:] = list(temp.iloc[:,2:].apply(lambda x: adf_test(np.log(x)), axis=0))")
    exec("method_" + str(n) + ".loc['diff',:] = list(temp.iloc[:,2:].apply(lambda x: adf_test(x.diff()[1:]), axis=0))")
    exec("method_" + str(n) + ".loc['log_diff',:] = list(temp.iloc[:,2:].apply(lambda x: adf_test(np.log(x).diff()[1:]), axis=0))")
    n+=1

In [ ]:
for i in sorted(set(temp1['emd_nm'])):
    temp = temp1[temp1['emd_nm'] == i].reset_index(drop=True)
    print(i)
    print(adf_test(np.log(temp['em_g']).diff()[1:]))

In [ ]:
def method(x):
    if (x[0] < 0.05):
        return 'raw'
    elif (x[1] < 0.05):
        return 'diff'
    elif (x[2] < 0.05):
        return 'log'
    elif (x[3] < 0.05):
        return 'log_diff'
    else:
        return np.nan

In [ ]:
method_0.loc['method',:] = method_0.apply(lambda x: method(x), axis=0)

In [ ]:
for i in range(0, len(set(temp2_prep['emd_nm']))):
    exec("method_" + str(i) + ".loc['method',:] = method_" + str(i) + ".apply(lambda x: method(x), axis=0)")

In [ ]:
def trans(x, m):
    if m == 'raw':
        return x.shift(1)
    elif m == 'diff':
        return x.astype('float').diff().shift(1)
    elif m == 'log':
        return np.log(x.astype('float')).shift(1)
    elif m == 'log_diff':
        return np.log(x.astype('float')).diff().shift(1)
    else:
        return pd.Series(np.repeat(np.nan, len(x)))

In [ ]:
n=0
corr_result = pd.DataFrame(index = sorted(list(set(temp2_prep['emd_nm']))),
                          columns = temp_var)
prep_data = pd.DataFrame()
for i in sorted(set(temp2_prep['emd_nm'])):
    if (i in set(temp1['emd_nm'])):
        temp = temp1[temp1['emd_nm'] == i].reset_index(drop=True)
        temp['em_g'] = temp['em_g'].diff()
        temp_1 = temp2_prep[temp2_prep['emd_nm'] == i].reset_index(drop=True)
        temp_2 = temp_1.T.iloc[2:,:]
        exec("temp_2['method'] = method_"+ str(n) +".T['method']")
        temp_1.iloc[:,2:] = temp_2.apply(lambda x: trans(x[:-1],x.iloc[-1:].values), axis=1).T
        temp_3 = pd.merge(temp_1, temp[['year_month', 'em_g']], on='year_month', how='outer')
        temp_4 = temp_3.iloc[:,2:-1]
        temp_5 = temp_3.iloc[:,-1]
        corr_result.loc[i,:] = temp_4.apply(lambda x: x.corr(temp_5), axis=0)
        prep_data = pd.concat([prep_data, temp_3.iloc[:,:-1]], axis=0)
    n+=1

In [ ]:
corr_result[abs(corr_result)>0.3].to_csv('corr_result_final.csv', encoding = 'cp949')

In [ ]:
for i in sorted(set(prep_data['emd_nm'])):
    temp_1 = prep_data[prep_data['emd_nm'] == i]
    temp_2 = corr_result.loc[i, :]
    temp_1[temp[temp.isna() == True].index] = np.nan
    prep_data[prep_data['emd_nm'] == i] = temp_1

In [ ]:
prep_data_1 = pd.DataFrame()
for i in sorted(set(prep_data['emd_nm'])):
    temp_1 = prep_data[prep_data['emd_nm'] == i].reset_index(drop = True)
    temp_2 = temp1[temp1['emd_nm'] == i].reset_index(drop = True)
    temp_2['em_g'] = temp_2['em_g'].diff()
    prep_data_1 = pd.concat([prep_data_1, pd.merge(temp_1, temp_2[['year_month', 'em_g']], on='year_month', how='outer')])

In [ ]:
temp_1 = prep_data[prep_data['emd_nm'] == '건입동'].reset_index(drop = True)
temp_2 = temp1[temp1['emd_nm'] == '건입동'].reset_index(drop = True)

In [ ]:
temp_2['em_g'] = temp_2['em_g'].diff()

In [ ]:
card_result = pd.read_csv('card_result.csv')
long_foreign = pd.read_csv('foreign_result.csv')

In [ ]:
card_result = card_result.drop(['Unnamed: 0'], axis=1)

In [ ]:
card_result = card_result.sort_values(by=['emd_nm', 'mct_cat_nm', 'year_month']).reset_index(drop=True)

In [ ]:
card_data = pd.DataFrame()
card_data['emd_nm'] = np.nan
card_data['year_month'] = np.nan
card_data[list(sorted(set(card_result['mct_cat_nm'])))] = np.nan

for i in sorted(set(prep_data_1['emd_nm'])):
    temp_data = pd.DataFrame()
    temp_data['year_month'] = sorted(set(prep_data['year_month']))
    temp_data['emd_nm'] = i
    temp_data[sorted(set(card_result['mct_cat_nm']))] = np.nan
    temp = card_result[card_result['emd_nm'] == i].reset_index(drop=True)
    for j in sorted(set(temp['mct_cat_nm'])):
        temp_1 = temp[temp['mct_cat_nm'] == j].reset_index(drop = True)
        temp_data[j] = temp_1['use_amt2']
    card_data = pd.concat([card_data, temp_data], axis=0)

In [ ]:
card_data = card_data.reset_index(drop=True)

In [ ]:
long_foreign = long_foreign.drop(['Unnamed: 0'], axis=1)

In [ ]:
long_foreign = long_foreign.sort_values(by = ['emd_nm',  'nationality','year_month']).reset_index(drop=True)

In [ ]:
lf_data = pd.DataFrame()
lf_data['emd_nm'] = np.nan
lf_data['year_month'] = np.nan
lf_data[list(sorted(set(long_foreign['nationality'])))] = np.nan

for i in sorted(set(prep_data_1['emd_nm'])):
    temp_data = pd.DataFrame()
    temp_data['year_month'] = sorted(set(long_foreign['year_month']))
    temp_data['emd_nm'] = i
    temp_data[sorted(set(long_foreign['nationality']))] = np.nan
    temp = long_foreign[long_foreign['emd_nm'] == i].reset_index(drop=True)
    for j in sorted(set(long_foreign['nationality'])):
        temp_1 = temp[temp['nationality'] == j].reset_index(drop = True)
        temp_data[j] = temp_1['tot_pop_cnt2']
    lf_data = pd.concat([lf_data, temp_data], axis=0)

In [ ]:
lf_data = lf_data.reset_index(drop=True)

In [ ]:
prep_data_1 = prep_data_1.reset_index(drop=True)

In [ ]:
X=prep_data_1.iloc[:,:-1]
y = prep_data_1.iloc[:,-1]

In [ ]:
X = pd.concat([X, card_data.iloc[:,2:], lf_data.iloc[:,2:]], axis=1).reset_index(drop=True)
#X = pd.concat([X, lf_data.iloc[:,2:]], axis=1).reset_index(drop=True)
prep_data_2 = pd.concat([X, y], axis=1).reset_index(drop=True)

In [ ]:
t1 = pd.read_csv('jeju_temp.csv', encoding = 'cp949')
t2 = pd.read_csv('songsan_temp.csv', encoding= 'cp949')
t3 = pd.read_csv('gosan_temp.csv', encoding='cp949')
t4 = pd.read_csv('seogwipo_temp.csv', encoding='cp949')

In [ ]:
obs = pd.read_csv('행정동별 날씨 관측지.csv', encoding='cp949')

In [ ]:
X = prep_data_2.iloc[:, :-1]
y = prep_data_2.iloc[:, -1]

In [ ]:
X['avg_temp'] = np.nan
for i in sorted(set(prep_data_2['emd_nm'])):
    temp_spot = obs['spot'][obs[obs['emd_nm'] == i].index]
    
    if (temp_spot == '제주').sum():
        X['avg_temp'][prep_data_2['emd_nm'] == i] = list(t1['평균기온(°C)'])
    elif (temp_spot == '성산').sum():
        X['avg_temp'][prep_data_2['emd_nm'] == i] = list(t2['평균기온(°C)'])
    elif (temp_spot == '고산').sum():
        X['avg_temp'][prep_data_2['emd_nm'] == i] = list(t3['평균기온(°C)'])
    elif (temp_spot == '서귀포').sum():
        X['avg_temp'][prep_data_2['emd_nm'] == i] = list(t4['평균기온(°C)'])

In [ ]:
prep_data_2 = pd.concat([X,y], axis=1)

In [ ]:
short_foreign = pd.read_csv('단기체류변형값만.csv')

In [ ]:
short_foreign = short_foreign.drop(['Unnamed: 0'], axis=1)
short_foreign = short_foreign.sort_values(by = ['emd_nm',  'nationality','year_month']).reset_index(drop=True)
sf_data = pd.DataFrame()
sf_data['emd_nm'] = np.nan
sf_data['year_month'] = np.nan



for i in sorted(set(prep_data_1['emd_nm'])):
    temp_data = pd.DataFrame()
    temp_data['year_month'] = sorted(set(short_foreign['year_month']))
    temp_data['emd_nm'] = i
    temp = short_foreign[short_foreign['emd_nm'] == i].reset_index(drop=True)
    for j in sorted(set(short_foreign['nationality'])):
        temp_1 = temp[temp['nationality'] == j].reset_index(drop = True)
        temp_data['S-' + j] = temp_1['visit_pop_cnt2']
    sf_data = pd.concat([sf_data, temp_data], axis=0)

In [ ]:
sf_data = sf_data.reset_index(drop=True)

In [ ]:
sf_data

In [ ]:
X = prep_data_2.iloc[:,:-2]
y = prep_data_2.iloc[:,-2:]

In [ ]:
X = pd.concat([X, sf_data.iloc[:,2:]], axis=1).reset_index(drop=True)
prep_data_3 = pd.concat([X, y], axis=1).reset_index(drop=True)
prep_data_3

In [ ]:
r1 = pd.read_csv('jeju_rain.csv', encoding = 'cp949')
r2 = pd.read_csv('songsan_rain.csv', encoding = 'cp949')
r3 = pd.read_csv('gosan_rain.csv', encoding = 'cp949')
r4 = pd.read_csv('seogwipo_rain.csv', encoding = 'cp949')

r1

In [ ]:
X['rain'] = np.nan
for i in sorted(set(prep_data_2['emd_nm'])):
    temp_spot = obs['spot'][obs[obs['emd_nm'] == i].index]
    
    if (temp_spot == '제주').sum():
        X['rain'][prep_data_3['emd_nm'] == i] = list(r1['월합강수량(00~24h만)(mm)'])
    elif (temp_spot == '성산').sum():
        X['rain'][prep_data_3['emd_nm'] == i] = list(r2['월합강수량(00~24h만)(mm)'])
    elif (temp_spot == '고산').sum():
        X['rain'][prep_data_3['emd_nm'] == i] = list(r3['월합강수량(00~24h만)(mm)'])
    elif (temp_spot == '서귀포').sum():
        X['rain'][prep_data_3['emd_nm'] == i] = list(r4['월합강수량(00~24h만)(mm)'])

In [ ]:
prep_data_3['rain'] = X['rain']

In [ ]:
temp_y = prep_data_3['em_g']
temp_x = prep_data_3.drop(['em_g'], axis=1)


In [ ]:
prep_data_3 = pd.concat([temp_x, temp_y], axis=1)

In [ ]:
for i in sorted(set(prep_data_3['emd_nm'])):
    prep_data_3['avg_temp'][prep_data_3['emd_nm'] == i] = prep_data_3['avg_temp'][prep_data_3['emd_nm'] == i].shift(1)
    prep_data_3['rain'][prep_data_3['emd_nm'] == i] = prep_data_3['rain'][prep_data_3['emd_nm'] == i].shift(1)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import statsmodels.api as sm
for i in sorted(set(prep_data_3['emd_nm'])):
    temp = prep_data_3[prep_data_3['emd_nm'] == i].iloc[:, 2:].reset_index(drop=True)
    target = temp.iloc[:, -1]
    temp_1 = temp.iloc[:, :-1]
    
    n = target.isna().sum() + 1
    
    target = target.iloc[n:].reset_index(drop=True)
    temp_1 = temp_1.iloc[n:,:].reset_index(drop=True)
    temp_1 = temp_1.dropna(axis=1)
    std_df = StandardScaler().fit_transform(temp_1)
    std_df = pd.DataFrame(std_df, index=temp_1.index, columns=temp_1.columns)
    temp_1 = std_df
    pca = PCA(n_components=min(len(temp_1.columns), len(temp_1.index)))
    pca_array = pca.fit_transform(temp_1)
    pca_df = pd.DataFrame(pca_array, index=temp_1.index,
                      columns=[f"pca{num+1}" for num in range(min(len(temp_1.columns), len(temp_1.index)))])
    pca_df.head()
    result = pd.DataFrame({'설명가능한 분산 비율(고윳값)':pca.explained_variance_,
             '기여율':pca.explained_variance_ratio_},
            index=np.array([f"pca{num+1}" for num in range(min(len(temp_1.columns), len(temp_1.index)))]))

    result['누적기여율'] = result['기여율'].cumsum()
    print(i)
    #print(result)
    n = len(result['누적기여율']>0.95) - (result['누적기여율']>0.95).sum() + 1
    pca_x = PCA(n_components = n)
    pca_x_array = pca_x.fit_transform(temp_1)
    pca_df = pd.DataFrame(pca_x_array, index = temp_1.index,
                         columns=[f"pca{num+1}" for num in range(n)])
    pca_df = sm.add_constant(pca_df, has_constant = 'add')
    temp_loop = True
    while (temp_loop):
        model = sm.OLS(target, pca_df)
        fitted_model = model.fit()
        p = pd.DataFrame(fitted_model.pvalues).reset_index()
        if (p.iloc[1:,1] > 0.05).sum() == 0:
            temp_loop = False
        else:
            temp_drop = p[p.iloc[0:,1] == max(p.iloc[1:,1])].iloc[0]['index']
            pca_df = pca_df.drop([temp_drop], axis=1)
    print(i)
    model = sm.OLS(target, pca_df)
    fitted_model = model.fit()
    print(fitted_model.summary())